In [ ]:
!pip install pyspark

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark=SparkSession.builder.getOrCreate()

In [57]:
df = spark.read.format("csv")\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .load("/content/details/data")

In [ ]:
df.show(10)

+-------+----------+----------+--------+--------+---------+--------+
|Company|      Date|Close/Last|  Volume|    Open|     High|     Low|
+-------+----------+----------+--------+--------+---------+--------+
|   AAPL|07/17/2023|   $193.99|50520160| $191.90|  $194.32| $191.81|
|   AAPL|07/14/2023|   $190.69|41616240| $190.23|$191.1799| $189.63|
|   AAPL|07/13/2023|   $190.54|41342340| $190.50|  $191.19| $189.78|
|   AAPL|07-12-2023|   $189.77|60750250| $189.68|  $191.70| $188.47|
|   AAPL|07-11-2023|   $188.08|46638120| $189.16|  $189.30| $186.60|
|   AAPL|07-10-2023|   $188.61|59922160| $189.26|  $189.99|$187.035|
|   AAPL|07-07-2023|   $190.68|46815000| $191.41|  $192.67| $190.24|
|   AAPL|07-06-2023|   $191.81|45156010| $189.84|  $192.02| $189.20|
|   AAPL|07-05-2023|   $191.33|46920260|$191.565|  $192.98| $190.62|
|   AAPL|07-03-2023|   $192.46|31346600| $193.78|  $193.88| $191.76|
+-------+----------+----------+--------+--------+---------+--------+
only showing top 10 rows



In [ ]:
df.printSchema()

root
 |-- Company: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Close/Last: string (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Open: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)



In [ ]:
from pyspark.sql.functions import col, regexp_replace
from pyspark.sql.functions import col,date_format,to_date,max

In [ ]:
df = df.withColumn("Date", regexp_replace(col("Date"), "/", "-"))

In [ ]:
df.show()

+-------+----------+----------+---------+--------+---------+---------+
|Company|      Date|Close/Last|   Volume|    Open|     High|      Low|
+-------+----------+----------+---------+--------+---------+---------+
|   AAPL|07-17-2023|   $193.99| 50520160| $191.90|  $194.32|  $191.81|
|   AAPL|07-14-2023|   $190.69| 41616240| $190.23|$191.1799|  $189.63|
|   AAPL|07-13-2023|   $190.54| 41342340| $190.50|  $191.19|  $189.78|
|   AAPL|07-12-2023|   $189.77| 60750250| $189.68|  $191.70|  $188.47|
|   AAPL|07-11-2023|   $188.08| 46638120| $189.16|  $189.30|  $186.60|
|   AAPL|07-10-2023|   $188.61| 59922160| $189.26|  $189.99| $187.035|
|   AAPL|07-07-2023|   $190.68| 46815000| $191.41|  $192.67|  $190.24|
|   AAPL|07-06-2023|   $191.81| 45156010| $189.84|  $192.02|  $189.20|
|   AAPL|07-05-2023|   $191.33| 46920260|$191.565|  $192.98|  $190.62|
|   AAPL|07-03-2023|   $192.46| 31346600| $193.78|  $193.88|  $191.76|
|   AAPL|06-30-2023|   $193.97| 85213220| $191.63|  $194.48|  $191.26|
|   AA

In [ ]:
df = df.withColumn("Date", to_date(col("Date"), "MM-dd-yyyy"))
df.printSchema()

root
 |-- Company: string (nullable = true)
 |-- Date: date (nullable = true)
 |-- Close/Last: string (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Open: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)



In [ ]:
from pyspark.sql.functions import col, min, max
df.agg(min(col("Date"))).first()[0]

datetime.date(2013, 7, 18)

In [ ]:
df.agg(max(col("Date"))).first()[0]

datetime.date(2023, 7, 17)

In [ ]:
def remove_dollar_sign_and_convert_to_float(column):
    return regexp_replace(column, r'[\$,]', '').cast('float')

In [ ]:
for col in df.columns:
    if col not in ["Company", "Date", "Volume"]:
        df = df.withColumn(col, remove_dollar_sign_and_convert_to_float(col))

In [ ]:
df.show()

+-------+----------+----------+---------+-------+--------+--------+
|Company|      Date|Close/Last|   Volume|   Open|    High|     Low|
+-------+----------+----------+---------+-------+--------+--------+
|   AAPL|2023-07-17|    193.99| 50520160|  191.9|  194.32|  191.81|
|   AAPL|2023-07-14|    190.69| 41616240| 190.23|191.1799|  189.63|
|   AAPL|2023-07-13|    190.54| 41342340|  190.5|  191.19|  189.78|
|   AAPL|2023-07-12|    189.77| 60750250| 189.68|   191.7|  188.47|
|   AAPL|2023-07-11|    188.08| 46638120| 189.16|   189.3|   186.6|
|   AAPL|2023-07-10|    188.61| 59922160| 189.26|  189.99| 187.035|
|   AAPL|2023-07-07|    190.68| 46815000| 191.41|  192.67|  190.24|
|   AAPL|2023-07-06|    191.81| 45156010| 189.84|  192.02|   189.2|
|   AAPL|2023-07-05|    191.33| 46920260|191.565|  192.98|  190.62|
|   AAPL|2023-07-03|    192.46| 31346600| 193.78|  193.88|  191.76|
|   AAPL|2023-06-30|    193.97| 85213220| 191.63|  194.48|  191.26|
|   AAPL|2023-06-29|    189.59| 46347310| 189.08

In [ ]:
df.printSchema()

root
 |-- Company: string (nullable = true)
 |-- Date: date (nullable = true)
 |-- Close/Last: float (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Open: float (nullable = true)
 |-- High: float (nullable = true)
 |-- Low: float (nullable = true)



In [ ]:
dfopen=df.select("Company","Date","Open")

In [ ]:
dfopen.show()

+-------+----------+-------+
|Company|      Date|   Open|
+-------+----------+-------+
|   AAPL|2023-07-17|  191.9|
|   AAPL|2023-07-14| 190.23|
|   AAPL|2023-07-13|  190.5|
|   AAPL|2023-07-12| 189.68|
|   AAPL|2023-07-11| 189.16|
|   AAPL|2023-07-10| 189.26|
|   AAPL|2023-07-07| 191.41|
|   AAPL|2023-07-06| 189.84|
|   AAPL|2023-07-05|191.565|
|   AAPL|2023-07-03| 193.78|
|   AAPL|2023-06-30| 191.63|
|   AAPL|2023-06-29| 189.08|
|   AAPL|2023-06-28| 187.93|
|   AAPL|2023-06-27| 185.89|
|   AAPL|2023-06-26| 186.83|
|   AAPL|2023-06-23| 185.55|
|   AAPL|2023-06-22| 183.74|
|   AAPL|2023-06-21|  184.9|
|   AAPL|2023-06-20| 184.41|
|   AAPL|2023-06-16| 186.73|
+-------+----------+-------+
only showing top 20 rows



In [ ]:
from pyspark.sql.functions import split, col

In [ ]:
dfopen=dfopen.withColumn("Year", split(col("Date"), "-").getItem(0))
dfopen.show()

+-------+----------+-------+----+
|Company|      Date|   Open|Year|
+-------+----------+-------+----+
|   AAPL|2023-07-17|  191.9|2023|
|   AAPL|2023-07-14| 190.23|2023|
|   AAPL|2023-07-13|  190.5|2023|
|   AAPL|2023-07-12| 189.68|2023|
|   AAPL|2023-07-11| 189.16|2023|
|   AAPL|2023-07-10| 189.26|2023|
|   AAPL|2023-07-07| 191.41|2023|
|   AAPL|2023-07-06| 189.84|2023|
|   AAPL|2023-07-05|191.565|2023|
|   AAPL|2023-07-03| 193.78|2023|
|   AAPL|2023-06-30| 191.63|2023|
|   AAPL|2023-06-29| 189.08|2023|
|   AAPL|2023-06-28| 187.93|2023|
|   AAPL|2023-06-27| 185.89|2023|
|   AAPL|2023-06-26| 186.83|2023|
|   AAPL|2023-06-23| 185.55|2023|
|   AAPL|2023-06-22| 183.74|2023|
|   AAPL|2023-06-21|  184.9|2023|
|   AAPL|2023-06-20| 184.41|2023|
|   AAPL|2023-06-16| 186.73|2023|
+-------+----------+-------+----+
only showing top 20 rows



In [ ]:
dfopen= dfopen.filter(dfopen["Year"] ==2020)
dfopen.show()

+-------+----------+------+----+
|Company|      Date|  Open|Year|
+-------+----------+------+----+
|   AAPL|2020-12-31|134.08|2020|
|   AAPL|2020-12-30|135.58|2020|
|   AAPL|2020-12-29|138.05|2020|
|   AAPL|2020-12-28|133.99|2020|
|   AAPL|2020-12-24|131.32|2020|
|   AAPL|2020-12-23|132.16|2020|
|   AAPL|2020-12-22|131.61|2020|
|   AAPL|2020-12-21|125.02|2020|
|   AAPL|2020-12-18|128.96|2020|
|   AAPL|2020-12-17| 128.9|2020|
|   AAPL|2020-12-16|127.41|2020|
|   AAPL|2020-12-15|124.34|2020|
|   AAPL|2020-12-14| 122.6|2020|
|   AAPL|2020-12-11|122.43|2020|
|   AAPL|2020-12-10| 120.5|2020|
|   AAPL|2020-12-09|124.53|2020|
|   AAPL|2020-12-08|124.37|2020|
|   AAPL|2020-12-07|122.31|2020|
|   AAPL|2020-12-04| 122.6|2020|
|   AAPL|2020-12-03|123.52|2020|
+-------+----------+------+----+
only showing top 20 rows



In [ ]:
dfopen.createOrReplaceTempView('dfmax')

In [ ]:
dfmax = spark.sql("""SELECT Company, Date, Open FROM dfmax WHERE (Company, Open) IN
 (SELECT Company, MAX(Open) AS max_open FROM dfmax GROUP BY Company)""")

In [ ]:
dfmax.show()

+-------+----------+------+
|Company|      Date|  Open|
+-------+----------+------+
|   AAPL|2020-12-29|138.05|
|   SBUX|2020-12-31| 106.0|
|   MSFT|2020-09-03|229.27|
|   CSCO|2020-02-12| 49.43|
|   QCOM|2020-12-09| 158.6|
|   META|2020-08-27|300.16|
|   AMZN|2020-09-02|177.35|
|   TSLA|2020-12-31|233.33|
|    AMD|2020-12-17| 97.55|
|   NFLX|2020-07-13|567.98|
+-------+----------+------+



In [ ]:
df.show()

+-------+----------+----------+---------+-------+--------+--------+
|Company|      Date|Close/Last|   Volume|   Open|    High|     Low|
+-------+----------+----------+---------+-------+--------+--------+
|   AAPL|2023-07-17|    193.99| 50520160|  191.9|  194.32|  191.81|
|   AAPL|2023-07-14|    190.69| 41616240| 190.23|191.1799|  189.63|
|   AAPL|2023-07-13|    190.54| 41342340|  190.5|  191.19|  189.78|
|   AAPL|2023-07-12|    189.77| 60750250| 189.68|   191.7|  188.47|
|   AAPL|2023-07-11|    188.08| 46638120| 189.16|   189.3|   186.6|
|   AAPL|2023-07-10|    188.61| 59922160| 189.26|  189.99| 187.035|
|   AAPL|2023-07-07|    190.68| 46815000| 191.41|  192.67|  190.24|
|   AAPL|2023-07-06|    191.81| 45156010| 189.84|  192.02|   189.2|
|   AAPL|2023-07-05|    191.33| 46920260|191.565|  192.98|  190.62|
|   AAPL|2023-07-03|    192.46| 31346600| 193.78|  193.88|  191.76|
|   AAPL|2023-06-30|    193.97| 85213220| 191.63|  194.48|  191.26|
|   AAPL|2023-06-29|    189.59| 46347310| 189.08

In [ ]:
df1=df.select("Company","Date","Open","Close/Last")
df1.show()

+-------+----------+-------+----------+
|Company|      Date|   Open|Close/Last|
+-------+----------+-------+----------+
|   AAPL|2023-07-17|  191.9|    193.99|
|   AAPL|2023-07-14| 190.23|    190.69|
|   AAPL|2023-07-13|  190.5|    190.54|
|   AAPL|2023-07-12| 189.68|    189.77|
|   AAPL|2023-07-11| 189.16|    188.08|
|   AAPL|2023-07-10| 189.26|    188.61|
|   AAPL|2023-07-07| 191.41|    190.68|
|   AAPL|2023-07-06| 189.84|    191.81|
|   AAPL|2023-07-05|191.565|    191.33|
|   AAPL|2023-07-03| 193.78|    192.46|
|   AAPL|2023-06-30| 191.63|    193.97|
|   AAPL|2023-06-29| 189.08|    189.59|
|   AAPL|2023-06-28| 187.93|    189.25|
|   AAPL|2023-06-27| 185.89|    188.06|
|   AAPL|2023-06-26| 186.83|    185.27|
|   AAPL|2023-06-23| 185.55|    186.68|
|   AAPL|2023-06-22| 183.74|     187.0|
|   AAPL|2023-06-21|  184.9|    183.96|
|   AAPL|2023-06-20| 184.41|    185.01|
|   AAPL|2023-06-16| 186.73|    184.92|
+-------+----------+-------+----------+
only showing top 20 rows



In [ ]:
df1.printSchema()

root
 |-- Company: string (nullable = true)
 |-- Date: date (nullable = true)
 |-- Open: float (nullable = true)
 |-- Close/Last: float (nullable = true)



In [ ]:
df1 = df1.withColumn("Date", date_format("Date", "yyyy-MM"))
df1.show()

+-------+-------+-------+----------+
|Company|   Date|   Open|Close/Last|
+-------+-------+-------+----------+
|   AAPL|2023-07|  191.9|    193.99|
|   AAPL|2023-07| 190.23|    190.69|
|   AAPL|2023-07|  190.5|    190.54|
|   AAPL|2023-07| 189.68|    189.77|
|   AAPL|2023-07| 189.16|    188.08|
|   AAPL|2023-07| 189.26|    188.61|
|   AAPL|2023-07| 191.41|    190.68|
|   AAPL|2023-07| 189.84|    191.81|
|   AAPL|2023-07|191.565|    191.33|
|   AAPL|2023-07| 193.78|    192.46|
|   AAPL|2023-06| 191.63|    193.97|
|   AAPL|2023-06| 189.08|    189.59|
|   AAPL|2023-06| 187.93|    189.25|
|   AAPL|2023-06| 185.89|    188.06|
|   AAPL|2023-06| 186.83|    185.27|
|   AAPL|2023-06| 185.55|    186.68|
|   AAPL|2023-06| 183.74|     187.0|
|   AAPL|2023-06|  184.9|    183.96|
|   AAPL|2023-06| 184.41|    185.01|
|   AAPL|2023-06| 186.73|    184.92|
+-------+-------+-------+----------+
only showing top 20 rows



In [ ]:
df2 = df1.groupBy("Company", "Date").agg(max("Close/Last").alias("MaxClosingPrice"),min("Close/Last").alias("MinClosingPrice"))
df2.show()

+-------+-------+---------------+---------------+
|Company|   Date|MaxClosingPrice|MinClosingPrice|
+-------+-------+---------------+---------------+
|   AAPL|2018-08|        56.9075|         50.375|
|   AAPL|2015-11|        30.6425|         28.085|
|   SBUX|2019-02|           71.3|          67.58|
|   MSFT|2016-08|           58.3|          56.58|
|   MSFT|2014-01|          37.84|          34.98|
|   CSCO|2014-09|          25.22|          24.54|
|   QCOM|2017-11|          68.91|          53.46|
|    AMD|2014-12|           2.74|           2.47|
|   CSCO|2020-02|          49.93|          39.93|
|   QCOM|2020-05|          80.88|          75.64|
|   QCOM|2019-12|          89.05|          80.58|
|   TSLA|2014-02|        16.8667|         11.628|
|    AMD|2022-01|         150.24|          102.6|
|   NFLX|2019-12|          336.9|         293.12|
|   MSFT|2019-07|         141.34|         135.68|
|   MSFT|2019-03|         120.22|         110.39|
|   TSLA|2017-10|        23.9767|        21.3387|


In [ ]:
df2 = df2.orderBy("Date")
df2.show()

+-------+-------+---------------+---------------+
|Company|   Date|MaxClosingPrice|MinClosingPrice|
+-------+-------+---------------+---------------+
|   AMZN|2013-07|        15.6005|         14.947|
|   AAPL|2013-07|          16.19|        14.9639|
|   CSCO|2013-07|          25.86|          25.33|
|   TSLA|2013-07|         8.9747|         7.9353|
|   META|2013-07|         37.627|         25.881|
|   QCOM|2013-07|          64.61|         61.115|
|   MSFT|2013-07|          35.44|          31.39|
|   SBUX|2013-07|          36.68|         33.305|
|   NFLX|2013-07|        38.0586|        34.4714|
|    AMD|2013-07|           4.64|           3.63|
|   AMZN|2013-08|        15.2785|        14.0465|
|   QCOM|2013-08|          67.25|          65.21|
|   META|2013-08|          41.34|          36.56|
|   TSLA|2013-08|        11.2667|         8.9487|
|   MSFT|2013-08|          34.75|         31.393|
|   CSCO|2013-08|         26.377|          23.31|
|   SBUX|2013-08|         37.115|          35.08|


In [ ]:
df2=df2.withColumn("Month",split(df2['Date'],"-")[1])

In [ ]:
df2.show()

+-------+-------+---------------+---------------+-----+
|Company|   Date|MaxClosingPrice|MinClosingPrice|Month|
+-------+-------+---------------+---------------+-----+
|   AMZN|2013-07|        15.6005|         14.947|   07|
|   AAPL|2013-07|          16.19|        14.9639|   07|
|   CSCO|2013-07|          25.86|          25.33|   07|
|   TSLA|2013-07|         8.9747|         7.9353|   07|
|   META|2013-07|         37.627|         25.881|   07|
|   QCOM|2013-07|          64.61|         61.115|   07|
|   MSFT|2013-07|          35.44|          31.39|   07|
|   SBUX|2013-07|          36.68|         33.305|   07|
|   NFLX|2013-07|        38.0586|        34.4714|   07|
|    AMD|2013-07|           4.64|           3.63|   07|
|   AMZN|2013-08|        15.2785|        14.0465|   08|
|   QCOM|2013-08|          67.25|          65.21|   08|
|   META|2013-08|          41.34|          36.56|   08|
|   TSLA|2013-08|        11.2667|         8.9487|   08|
|   MSFT|2013-08|          34.75|         31.393

In [ ]:
from pyspark.sql import functions as sf

In [ ]:
df_high_close=df2.withColumn('Month_High_closing', sf.concat(sf.col('Month'),sf.lit('-'), sf.col('MaxClosingPrice')))
df_high_close=df_high_close.select("Company","Month_High_closing")
df_high_close.show()

+-------+------------------+
|Company|Month_High_closing|
+-------+------------------+
|   AMZN|        07-15.6005|
|   AAPL|          07-16.19|
|   CSCO|          07-25.86|
|   TSLA|         07-8.9747|
|   META|         07-37.627|
|   QCOM|          07-64.61|
|   MSFT|          07-35.44|
|   SBUX|          07-36.68|
|   NFLX|        07-38.0586|
|    AMD|           07-4.64|
|   AMZN|        08-15.2785|
|   QCOM|          08-67.25|
|   META|          08-41.34|
|   TSLA|        08-11.2667|
|   MSFT|          08-34.75|
|   CSCO|         08-26.377|
|   SBUX|         08-37.115|
|   NFLX|        08-41.1214|
|    AMD|           08-3.82|
|   AAPL|        08-18.1336|
+-------+------------------+
only showing top 20 rows



In [ ]:
df_low_close=df2.withColumn('Month_low_closing', sf.concat(sf.col('Month'),sf.lit('-'), sf.col('MinClosingPrice')))
df_low_close=df_low_close.select("Company","Month_low_closing")
df_low_close.show()

+-------+-----------------+
|Company|Month_low_closing|
+-------+-----------------+
|   AMZN|        07-14.947|
|   AAPL|       07-14.9639|
|   CSCO|         07-25.33|
|   TSLA|        07-7.9353|
|   META|        07-25.881|
|   QCOM|        07-61.115|
|   MSFT|         07-31.39|
|   SBUX|        07-33.305|
|   NFLX|       07-34.4714|
|    AMD|          07-3.63|
|   AMZN|       08-14.0465|
|   QCOM|         08-65.21|
|   META|         08-36.56|
|   TSLA|        08-8.9487|
|   MSFT|        08-31.393|
|   CSCO|         08-23.31|
|   SBUX|         08-35.08|
|   NFLX|       08-35.1686|
|    AMD|          08-3.27|
|   AAPL|       08-16.2304|
+-------+-----------------+
only showing top 20 rows



In [ ]:
df2.show()

+-------+-------+---------------+---------------+-----+
|Company|   Date|MaxClosingPrice|MinClosingPrice|Month|
+-------+-------+---------------+---------------+-----+
|   AMZN|2013-07|        15.6005|         14.947|   07|
|   AAPL|2013-07|          16.19|        14.9639|   07|
|   CSCO|2013-07|          25.86|          25.33|   07|
|   TSLA|2013-07|         8.9747|         7.9353|   07|
|   META|2013-07|         37.627|         25.881|   07|
|   QCOM|2013-07|          64.61|         61.115|   07|
|   MSFT|2013-07|          35.44|          31.39|   07|
|   SBUX|2013-07|          36.68|         33.305|   07|
|   NFLX|2013-07|        38.0586|        34.4714|   07|
|    AMD|2013-07|           4.64|           3.63|   07|
|   AMZN|2013-08|        15.2785|        14.0465|   08|
|   QCOM|2013-08|          67.25|          65.21|   08|
|   META|2013-08|          41.34|          36.56|   08|
|   TSLA|2013-08|        11.2667|         8.9487|   08|
|   MSFT|2013-08|          34.75|         31.393

In [ ]:
df3=df2.select("Company","Date","MaxClosingPrice")

In [ ]:
df3.show()

+-------+-------+---------------+
|Company|   Date|MaxClosingPrice|
+-------+-------+---------------+
|   AMZN|2013-07|        15.6005|
|   AAPL|2013-07|          16.19|
|   CSCO|2013-07|          25.86|
|   TSLA|2013-07|         8.9747|
|   META|2013-07|         37.627|
|   QCOM|2013-07|          64.61|
|   MSFT|2013-07|          35.44|
|   SBUX|2013-07|          36.68|
|   NFLX|2013-07|        38.0586|
|    AMD|2013-07|           4.64|
|   AMZN|2013-08|        15.2785|
|   QCOM|2013-08|          67.25|
|   META|2013-08|          41.34|
|   TSLA|2013-08|        11.2667|
|   MSFT|2013-08|          34.75|
|   CSCO|2013-08|         26.377|
|   SBUX|2013-08|         37.115|
|   NFLX|2013-08|        41.1214|
|    AMD|2013-08|           3.82|
|   AAPL|2013-08|        18.1336|
+-------+-------+---------------+
only showing top 20 rows



In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

In [ ]:
df3 = df3.withColumn("rank", F.rank().over(Window.partitionBy("Date").orderBy(F.desc("MaxClosingPrice"))))

In [ ]:
df3.show()

+-------+-------+---------------+----+
|Company|   Date|MaxClosingPrice|rank|
+-------+-------+---------------+----+
|   QCOM|2013-07|          64.61|   1|
|   NFLX|2013-07|        38.0586|   2|
|   META|2013-07|         37.627|   3|
|   SBUX|2013-07|          36.68|   4|
|   MSFT|2013-07|          35.44|   5|
|   CSCO|2013-07|          25.86|   6|
|   AAPL|2013-07|          16.19|   7|
|   AMZN|2013-07|        15.6005|   8|
|   TSLA|2013-07|         8.9747|   9|
|    AMD|2013-07|           4.64|  10|
|   QCOM|2013-08|          67.25|   1|
|   META|2013-08|          41.34|   2|
|   NFLX|2013-08|        41.1214|   3|
|   SBUX|2013-08|         37.115|   4|
|   MSFT|2013-08|          34.75|   5|
|   CSCO|2013-08|         26.377|   6|
|   AAPL|2013-08|        18.1336|   7|
|   AMZN|2013-08|        15.2785|   8|
|   TSLA|2013-08|        11.2667|   9|
|    AMD|2013-08|           3.82|  10|
+-------+-------+---------------+----+
only showing top 20 rows



In [ ]:
df3 = df3.where(F.col("rank") == 1).drop("rank")
df3.show()

+-------+-------+---------------+
|Company|   Date|MaxClosingPrice|
+-------+-------+---------------+
|   QCOM|2013-07|          64.61|
|   QCOM|2013-08|          67.25|
|   QCOM|2013-09|          70.09|
|   QCOM|2013-10|          69.49|
|   QCOM|2013-11|          73.65|
|   QCOM|2013-12|          74.25|
|   QCOM|2014-01|          75.87|
|   QCOM|2014-02|          76.44|
|   QCOM|2014-03|          79.28|
|   QCOM|2014-04|          81.32|
|   QCOM|2014-05|          80.46|
|   QCOM|2014-06|          80.48|
|   QCOM|2014-07|           81.6|
|   QCOM|2014-08|          77.01|
|   META|2014-09|          79.04|
|   META|2014-10|          80.77|
|   QCOM|2014-11|          78.24|
|   META|2014-12|          81.45|
|   META|2015-01|          78.45|
|   META|2015-02|          80.41|
+-------+-------+---------------+
only showing top 20 rows



In [ ]:
df3=df3.withColumn("Month",split(df3['Date'],"-")[1])

In [ ]:
df3.show()

+-------+-------+---------------+-----+
|Company|   Date|MaxClosingPrice|Month|
+-------+-------+---------------+-----+
|   QCOM|2013-07|          64.61|   07|
|   QCOM|2013-08|          67.25|   08|
|   QCOM|2013-09|          70.09|   09|
|   QCOM|2013-10|          69.49|   10|
|   QCOM|2013-11|          73.65|   11|
|   QCOM|2013-12|          74.25|   12|
|   QCOM|2014-01|          75.87|   01|
|   QCOM|2014-02|          76.44|   02|
|   QCOM|2014-03|          79.28|   03|
|   QCOM|2014-04|          81.32|   04|
|   QCOM|2014-05|          80.46|   05|
|   QCOM|2014-06|          80.48|   06|
|   QCOM|2014-07|           81.6|   07|
|   QCOM|2014-08|          77.01|   08|
|   META|2014-09|          79.04|   09|
|   META|2014-10|          80.77|   10|
|   QCOM|2014-11|          78.24|   11|
|   META|2014-12|          81.45|   12|
|   META|2015-01|          78.45|   01|
|   META|2015-02|          80.41|   02|
+-------+-------+---------------+-----+
only showing top 20 rows



In [ ]:
df3.select("Company").distinct().show()

+-------+
|Company|
+-------+
|   QCOM|
|   META|
|   TSLA|
|   NFLX|
|   MSFT|
+-------+



In [ ]:
df3.createOrReplaceTempView("df3")

In [ ]:
repationtions = spark.sql( """
    SELECT Company, Month, Count(*) as Repetitions
    FROM df3
    GROUP BY Company, Month
    ORDER BY Month, Repetitions DESC
""")

In [ ]:
repationtions.show()

+-------+-----+-----------+
|Company|Month|Repetitions|
+-------+-----+-----------+
|   NFLX|   01|          8|
|   META|   01|          1|
|   QCOM|   01|          1|
|   NFLX|   02|          7|
|   META|   02|          2|
|   QCOM|   02|          1|
|   NFLX|   03|          7|
|   META|   03|          2|
|   QCOM|   03|          1|
|   NFLX|   04|          7|
|   META|   04|          2|
|   QCOM|   04|          1|
|   NFLX|   05|          7|
|   QCOM|   05|          1|
|   TSLA|   05|          1|
|   META|   05|          1|
|   NFLX|   06|          7|
|   MSFT|   06|          1|
|   META|   06|          1|
|   QCOM|   06|          1|
+-------+-----+-----------+
only showing top 20 rows



In [ ]:
repationtions.createOrReplaceTempView("repationtions")

In [ ]:
maxrepetitions = spark.sql("""
    SELECT Company, Month, Repetitions
    FROM (
        SELECT Company, Month, Repetitions,
               RANK() OVER (PARTITION BY Month ORDER BY Repetitions DESC) as rank
        FROM repationtions
    ) ranked
    WHERE rank = 1
""")


In [ ]:
maxrepetitions.show()

+-------+-----+-----------+
|Company|Month|Repetitions|
+-------+-----+-----------+
|   NFLX|   01|          8|
|   NFLX|   02|          7|
|   NFLX|   03|          7|
|   NFLX|   04|          7|
|   NFLX|   05|          7|
|   NFLX|   06|          7|
|   NFLX|   07|          7|
|   NFLX|   08|          6|
|   NFLX|   09|          6|
|   NFLX|   10|          7|
|   NFLX|   11|          7|
|   NFLX|   12|          8|
+-------+-----+-----------+

